In [ ]:
# 일간 데이터를 월별 데이터로 구조 변경
# 월 - [일자별 데이터] : dictionary 형식

In [2]:
# 1. 전체 데이터 읽기
import csv

with open("data-files/seoul-temperature.csv") as f:
    reader = csv.reader(f)

    rows = []
    for row in reader:
        row[0] = row[0].strip("\t")
        rows.append(row)
        
source_rows = rows.copy()

In [5]:
rows[1][0][5:7]

'10'

In [7]:
# 월별로 데이터 재구성
data_by_month = {}
for row in rows:
    month = row[0][5:7] # '2023-03-08' -> '03'
    if month in data_by_month: # dict에 month 키가 있는지 확인
        data_by_month[month].append(row) # 있으면 추가
    else:
        data_by_month[month] = [row] # 없으면 새로 생성

In [9]:
from collections import defaultdict # dict의 초기 값 자동 관리

data_by_month2 = defaultdict(list) # list 형식을 값으로 사용하는 defaultdict
for row in rows:
    month = row[0][5:7] # '2023-03-08' -> '03'
    data_by_month2[month].append(row)
   

In [11]:
print( data_by_month["01"][:5] )
print( data_by_month2["01"][:5] )

[['1908-01-01', '108', '-5.9', '-9.1', '-1.3'], ['1908-01-02', '108', '-7.7', '-13.9', '-3'], ['1908-01-03', '108', '-6.2', '-10.8', '-2.9'], ['1908-01-04', '108', '-7.5', '-12.8', '-1.7'], ['1908-01-05', '108', '-4.8', '-11.3', '-0.1']]
[['1908-01-01', '108', '-5.9', '-9.1', '-1.3'], ['1908-01-02', '108', '-7.7', '-13.9', '-3'], ['1908-01-03', '108', '-6.2', '-10.8', '-2.9'], ['1908-01-04', '108', '-7.5', '-12.8', '-1.7'], ['1908-01-05', '108', '-4.8', '-11.3', '-0.1']]


In [17]:
# "05"월의 (평균기온의) 평균
# data_by_month["13"] # "13"에 해당하는 키가 없으면 오류
data = data_by_month.get("03", None)
if data == None:
    print("데이터가 없습니다.")
else:
    sum_value = 0
    cnt = 0
    for row in data:
        if row[2] != '':
            sum_value += float(row[2])
            cnt += 1

    print(sum_value / cnt)

4.5157849338700435
